In [1]:
from models.CNN_mutlitaper import ConvNetMultitaper
from pre_processing import multitaper
from data_loader.data_loaders import MultiTaperSet
from trainer.trainer import Trainer

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import pandas as pd
import seaborn as sns


## Prepare Cuda


In [ ]:
if torch.cuda.is_available():
  device = 'cuda'
else:
    device = 'cpu'
print(device)

## Create MultiTapers

In [2]:
multitaper.compute_tapers()

load data
tapers already computed


()

## Train CNN + Multitaper Model

In [3]:
CNN_taper_model = ConvNetMultitaper()
CNN_taper_model = CNN_taper_model.to(device)

In [4]:
multitaper_train_set = MultiTaperSet(device = device)
multitaper_test_set = MultiTaperSet(device=device, features_eeg_path = './data/pre_processed_data/Multitaper_eeg_test.npy', features_position_path = './data/pre_processed_data/Multitaper_position_test.npy')

In [5]:
train_ratio = 0.9
# split train and validation set
data_set = multitaper_train_set
train_len = int(len(data_set)*train_ratio)
train_set, validation_set = torch.utils.data.random_split(data_set, [train_len, len(data_set) - train_len])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
validationloader = torch.utils.data.DataLoader(validation_set, batch_size=len(validation_set), shuffle=True)

In [6]:
optimizer = torch.optim.Adam(CNN_taper_model.parameters())
trainer = Trainer(CNN_taper_model, nn.CrossEntropyLoss(), optimizer,trainloader, device, valid_data_loader = validationloader, class_weights=torch.Tensor([8.081897,22.222222, 2.756846, 3.765060, 4.927727]))

c:\Users\33631\Documents\Etudes\centrale 3A\Deep Learning\deep_learning_project\trainer\trainer.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.double)


In [7]:
loss_list = []
accuracy_list = []
for epoch in range(0, 25):
    loss, accuracy = trainer.train_epoch()
    loss_list.append(loss)
    accuracy_list.append(accuracy)

100%|██████████| 211/211 [00:58<00:00,  3.58it/s]


RuntimeError: CUDA out of memory. Tried to allocate 746.00 MiB (GPU 0; 2.00 GiB total capacity; 492.73 MiB already allocated; 515.19 MiB free; 524.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF